## Applying different strategy of triplet loss

In [0]:
LOCAL = False

In [0]:
%load_ext autoreload
%autoreload 2
%load_ext skip_cell

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
%%skip $LOCAL
#Mounting the drive

import zipfile
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
# %%skip $LOCAL

!cp -a "/content/drive/My Drive/triplets/" .

Setting up tensorboard for PyTorch in Colab

# Imports

In [0]:
import copy
import random
import time
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import torch
from torch import optim
from torch import nn
from torch.utils.data import random_split
from torch.utils.data.sampler import BatchSampler
from torchvision import transforms
from torchvision.datasets import SVHN
from torchvision.models import resnet18
from typing import Union
from PIL import Image

In [0]:
from triplets.datasets import TripletSVHN
from triplets.losses import TripletLoss, TripletSoftLoss, BatchAllTripletLoss, BatchHardTripletLoss, OnlineTripletLoss
from triplets.metrics import mean_average_precision
from triplets.nets import TripletNet
from triplets.train import train
from triplets.extractor import FeatureExtractor
from triplets.samplers import BalancedBatchSampler
from triplets.selectors import AllTripletSelector,HardestNegativeTripletSelector, RandomNegativeTripletSelector, \
                               SemihardNegativeTripletSelector # Strategies for selecting triplets within a minibatch
from triplets.utils import freeze_layers
from triplets.visualisation import plot_grad_flow

In [0]:
n_features = 512
n_classes = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
SEED = 100
validation_split = 0.2
shuffle_dataset = True

In [0]:
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

In [0]:
dataset = SVHN(root = 'data/', download=True, split='train')
train_size = int(0.8*len(dataset))
valid_size = len(dataset) - train_size
dataset_train, dataset_valid = random_split(dataset, [train_size, valid_size])
dataset_test = SVHN(root = 'data/', download=True, split='test');

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat


# Setting parameters for training model with triplet loss

In [0]:
batch_size = 32
num_triplets = 1
epochs = 20

In [0]:
model_base = resnet18(pretrained=True)
model_base.eval();

Defining extractor using custom class to extract features from last cnn pretrained layer.

In [0]:
extractor = FeatureExtractor(model=model_base, n_remove_layers=1, n_features=n_features, device=device)
extracted_resnet = extractor.prepare_model()
#Freezing all but two last layers
extracted_resnet = freeze_layers(extracted_resnet, 2)

In [0]:
preprocess = transforms.Compose([            
 transforms.Resize(256),                    
 transforms.CenterCrop(224),                
 transforms.ToTensor(),                     
 transforms.Normalize(                      
 mean=[0.485, 0.456, 0.406],                
 std=[0.229, 0.224, 0.225]                  
 )])

## Training with softmax triplet loss

Taken fron the paper titled "DEEP METRIC LEARNING USING TRIPLET NETWORK"




<br>
https://arxiv.org/pdf/1412.6622.pdf

In [0]:
triplet_train= TripletSVHN(dataset, dataset_train.indices, dataset_valid.indices,
                            preprocess, 'train', SEED)
triplet_valid = TripletSVHN(dataset, dataset_train.indices, dataset_valid.indices,
                            preprocess, 'val', SEED)

In [0]:
dataloader_train = torch.utils.data.DataLoader(triplet_train, batch_size=batch_size)
dataloader_valid = torch.utils.data.DataLoader(triplet_valid, batch_size=batch_size)
dataloaders = {'train': dataloader_train, 'val': dataloader_valid}

In [0]:
model = TripletNet(extracted_resnet)
criterion = TripletSoftLoss()
# Observe that all parameters are being optimized
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Decay LR by a factor of 0.1 every 7 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [0]:
train(model, dataloaders, criterion, optimizer, scheduler, epochs, device)

## Training with triplet loss provided FaceNet paper

We are going to use custom triplet dataset, which provides possibility of testing model on fixed sample and creates valid triplets in every iteration.

In [0]:
triplet_train= TripletSVHN(dataset, dataset_train.indices, dataset_valid.indices,
                            preprocess, 'train', SEED)
triplet_valid = TripletSVHN(dataset, dataset_train.indices, dataset_valid.indices,
                            preprocess, 'val', SEED)

In [0]:
dataloader_train = torch.utils.data.DataLoader(triplet_train, batch_size=batch_size)
dataloader_valid = torch.utils.data.DataLoader(triplet_valid, batch_size=batch_size)
dataloaders = {'train': dataloader_train, 'val': dataloader_valid}

In [0]:
model = TripletNet(extracted_resnet)
criterion = TripletLoss()
# Observe that all parameters are being optimized
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Decay LR by a factor of 0.1 every 7 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [0]:
train(model, dataloaders, criterion, optimizer, scheduler, epochs, device)

## Adding hard mining

In order to improve possible triplet selection we are going to use online triplet mining. It allows us to compute possible triplets in every iteration of the dataset, constructing $B^3$ triplets out of computed $B$ **embeddings**. Most of them are not relevant and we are going to use two strategies from which one can select possible triplets. In order to improve possible triplet selection we are going to use online triplet mining. Suppose that you have a batch of faces as input of size $B = P K$ , composed of $P$ different persons with $K$ images each. A typical value is $K = 4$ . The two strategies are:

**batch all**: select all the valid triplets, and average the loss on the hard and semi-hard triplets. a crucial point here is to not take into account the easy triplets (those with loss 0 ), as averaging on them would make the overall loss very small this produces a total of $P K ( K − 1 ) ( P K − K )$ triplets $P K$ anchors, $K − 1$ possible positives per anchor, $P K − K$ possible negatives) 
<br>
**batch hard**: for each anchor, select the hardest positive (biggest distance $d ( a , p ) )$ and the hardest negative among the batch this produces $P K$ triplets the selected triplets are the hardest among the batch.
<br>
[1] https://omoindrot.github.io/triplet-loss


In [0]:
extractor = FeatureExtractor(model=model_base, n_remove_layers=1, n_features=n_features, device=device)
extracted_resnet = extractor.prepare_model()
extracted_resnet = freeze_layers(extracted_resnet, 2)

In [0]:
train_batch_sampler = BalancedBatchSampler(dataset.labels[dataset_train.indices], n_classes=10, n_samples=10)

In [0]:
dataset = SVHN(root = 'data/', download=True, split='train', transform=preprocess)
train_size = int(0.8*len(dataset))
valid_size = len(dataset) - train_size
dataset_train, dataset_valid = random_split(dataset, [train_size, valid_size])
dataset_test = SVHN(root = 'data/', download=True, split='test', transform=preprocess);

Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat


In [0]:
# We'll create mini batches by sampling labels that will be present in the mini batch and number of examples from each class
train_batch_sampler = BalancedBatchSampler(dataset.labels[dataset_train.indices], n_classes=10, n_samples=25)
valid_batch_sampler = BalancedBatchSampler(dataset.labels[dataset_valid.indices], n_classes=10, n_samples=25)

online_train_loader = torch.utils.data.DataLoader(dataset_train, batch_sampler=train_batch_sampler)
online_valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_sampler=valid_batch_sampler)

margin = 1.
lr = 1e-3
optimizer = optim.Adam(extracted_resnet.parameters(), lr=lr, weight_decay=1e-4)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
n_epochs = 20
log_interval = 50

In [0]:
dataloaders = {'train': online_train_loader, 'val': online_valid_loader}

In [0]:
criterion = OnlineTripletLoss(margin, SemihardNegativeTripletSelector(margin))
# Observe that all parameters are being optimized
optimizer = optim.Adam(extracted_resnet.parameters(), lr=0.001)
# Decay LR by a factor of 0.1 every 7 epochs
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

# Optimizing the margin

I dedcided to use the loss function with online triplet mining technique in order to converge faster to desirable solution. 

* The first step was to calculate loss on both training and validation set, choosing model with lowest validation loss. 
* After that the kNN model was trained on the features obtained from the training with hyperparameter (number of neighbors) found in previous optimilisation (done directly on CNN codes)
* One can argue that this strategy is not optimal - we probably should tune both margin and number of neighors at the same time (for example using RandomSearch or BayesianOptimisation) however due to lack of training resourches we are going to stay with this grid search strategy.

In [0]:
epochs=20
margins = [0.01, 0.005, 0.1, 0.5, 1, 5]

In [0]:
for margin in margins:
    criterion = OnlineTripletLoss(margin, AllTripletSelector())
    semihard_trained_net, semihard_total_loss_train, semihard_total_loss_val = \
        train(extracted_resnet, dataloaders, criterion, optimizer, scheduler, epochs, 
        device, model_name='hardmine_all')

In [0]:
extractor = FeatureExtractor(model=semihard_trained_net, n_remove_layers=0, n_features=n_features, device=device)

In [0]:
clf = KNeighborsClassifier(n_neighbors=25)
features = extractor.extract_features(dataset_train)